## Best shit in the world


In [1]:
import sys
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image
import logging

from bokeh.io import output_file, show,output_notebook 
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import DatetimeTickFormatter

from bokeh.models import HoverTool, RangeTool, Slider

from bokeh.models.tools import *
from bokeh.models.annotations import Title
from bokeh.models import CustomJS
from bokeh.layouts import gridplot, grid, column

%matplotlib qt 

In [2]:
sys.path.insert(0,"..") # Adds higher directory to python modules path.

from traphing.data_classes import Velas, Symbol, Portfolio
from traphing.utils import Timeframes, unwrap
import traphing.utils as ul
from traphing.graph.Gl import gl

In [3]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


In [4]:
output_notebook()

Loading BokehJS ...

# Load the trading data and create a currency portfolio

In [5]:
portfolio_id = "Manu's winter portfolio"
symbol_names_list = []
timeframes_list = [Timeframes.M15, Timeframes.D1]

dataSource =  "MQL5"  # Hanseatic  FxPro GCI Yahoo
[storage_folder, updates_folder] = ul.get_foldersData(source = dataSource, rrf = "../../../" )
folder_images = "./images/python_generated/currencies/"

portfolio = Portfolio(portfolio_id, symbol_names_list, timeframes_list)
symbols_properties_df = Symbol.load_symbols_properties_csv(storage_folder)
all_symbol_names_list = symbols_properties_df.index.to_list()

## Empty portfolio for getting the desired available exchanges
all_portfolio = Portfolio("all currencies", all_symbol_names_list, [])

currency = "USD"
symbol_names_list = all_portfolio.get_related_forex_currencies(currency)

portfolio = Portfolio("USD related currencies", symbol_names_list, timeframes_list)
portfolio.load_data_from_csv(storage_folder)

start_time = dt.datetime(2019,7,20); end_time = dt.datetime(2019,7,25)
portfolio.set_time_interval(start_time,end_time)

""" Subselect portfolio"""
symbol_names_list = portfolio.get_non_empty(timeframes_list[0])
portfolio = portfolio.get_subportfolio(symbol_names_list )
if(0):
    portfolio.estimate_symbols_market_hours(timeframes_list[0])
    symbol_names_list = portfolio.get_currencies_with_market_hours(open_time = dt.time(0), close_time = dt.time(0))
    portfolio = portfolio.get_subportfolio(symbol_names_list)

currencies_df = portfolio.map_timeframe(timeframe = timeframes_list[0], method_name = "series", name = "Close")

In [6]:
print ("Loaded currencies:", symbol_names_list)

Loaded currencies: ['EURUSD', 'GBPUSD', 'USDCHF', 'USDJPY', 'USDCNH', 'USDRUB', 'AUDUSD', 'NZDUSD', 'USDCAD', 'USDSEK', 'USDHKD', 'USDSGD', 'USDMXN', 'USDNOK', 'USDDKK', 'USDTRY', 'USDZAR', 'USDCZK', 'USDHUF', 'USDPLN', 'USDRUR', 'XAUUSD', 'XAGUSD']


## Bokeh styling functions


In [7]:
def set_title(p, text, color = "olive", font = "times",style = "italic",size = "30px" ):
    t = Title()
    t.text = text
    t.text_color = color
    t.text_font = font
    t.text_font_style = style
    t.text_font_size = size
    
    p.title = t

def set_background(p, color):
    p.background_fill_color = color

# Hover tool!

In [8]:
# Easy not used in the end options
hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("(x,y)", "($x, $y)"),
            ("currency", "@currency"),
        ]
    )

In [9]:
hover = HoverTool(
        tooltips="""
        <div>
            <div>
                <span style="font-size: 17px; font-weight: bold;">@desc_left</span>
                <span style="font-size: 15px; color: #966;">[$index]/span>
            </div>
            <div>
                <span style="font-size: 15px;">Location</span>
                <span style="font-size: 10px; color: #696;">(@timestamp, $y)</span>
            </div>
        </div>
        """
    )

# Configuration of the Figure options:

In [10]:
width, height = 400, 300
x_axis_type="datetime" # datetime
toolbar_location = "above"
tools = ["pan,wheel_zoom,box_zoom,reset, box_select"]

background_color = "black"
line_color = "white"

### Design column data for placing more cleverly

In [11]:
timestamps = currencies_df.index
values = currencies_df["USDCHF"]
values2 = currencies_df["EURUSD"]

source_left = ColumnDataSource(
    data = dict(
        x =  timestamps,
        y = values,
        currency = [values.name] * timestamps.size,
        timestamp = timestamps.strftime('%B %d, %Y, %r')
    ),
    name = values.name
)

source_right = ColumnDataSource(
    data = dict(
        x =  timestamps,
        y = values2,
        currency = [values2.name] * timestamps.size,
        timestamp = timestamps.strftime('%B %d, %Y, %r')
    ),
    name = values2.name
)

source_all = ColumnDataSource(
    data = dict(
        x =  timestamps,
        y_left = values,
        y_left_original = values,
        y_right = values2,
        desc_left = [values.name] * timestamps.size,
        desc_right = [values2.name] * timestamps.size,
        timestamp = timestamps.strftime('%B %d, %Y, %r')
    ),
    name = values2.name
)


# Plot this shit

Let us select the range explicitly because Bokeh has bugs on it.

In [12]:
y_range = (np.min(values), np.max(values))
p_left = figure(plot_width = width, plot_height=height, tools = tools,
             x_axis_type=x_axis_type, toolbar_location=toolbar_location,
                  x_range=(timestamps[0], timestamps[100]),
               y_range=y_range)

# Make them share the x_range for theRange tool
p_right = figure(plot_width = width, plot_height=height, tools = tools,
             x_axis_type=x_axis_type, toolbar_location=toolbar_location,
                  x_range= p_left.x_range)

p_grid_list = [[p_left, p_right]]
# output_file("circle.html")

In [13]:
y_range

(0.98049, 0.99137)

In [14]:
if 0:
    p_left.scatter("x", "y",
             color=line_color, line_width=1,
             legend= source_left.name, source = source_left)

    p_right.scatter("x", "y",
             color=line_color, line_width=1,
             legend= source_right.name, source = source_right)
else:
    """ SHARED SELECTION """
    drawings_1 = p_left.scatter("x", "y_left",
             color=line_color, line_width=1,
             legend= source_left.name, source = source_all)
    if 1:
        caca = p_left.scatter("x", "y_left_original",
                 color="blue", line_width=1,
                 legend= source_left.name, source = source_all)
        # p_left.y_range.renderers = [drawings_1, caca]
        
    p_right.scatter("x", "y_right",
             color=line_color, line_width=1,
             legend= source_right.name, source = source_all)

In [15]:
# fig.xaxis[0].formatter = DatetimeTickFormatter()
for p_row in p_grid_list:
    for p in p_row:
        set_title(p, "Basic showing of the currency", size = "20px")
        set_background(p,background_color)

## Range tool

We add a new plot to which we will add the RangeTool. In the plot we give the x_range object of the chart we want to modify.
- original_p.x_range neded.
- We also share the x_range accross differen plots by giving it as a parameter in Figure() function.

In [16]:
p_range_tool = figure(title="Drag the middle and edges of the selection box to change the range above",
                plot_height=130, plot_width=width, y_range=p_left.y_range,
                x_axis_type="datetime", y_axis_type=None,
                tools="", toolbar_location=None, background_fill_color="#efefef")

range_tool = RangeTool(x_range=p_left.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

p_range_tool.line('x', 'y_left', source=source_all)
p_range_tool.ygrid.grid_line_color = None
p_range_tool.add_tools(range_tool)
p_range_tool.toolbar.active_multi = range_tool


Slider

We can add an Slider that is going to include a JS funct that will redraw the plot when triggered. For that we need to:
 - Create the Slider object: 
 - callback function that will be called when the Slider is changed
 - appending the callback to the slider 

In [17]:
amp_slider = Slider(start=0.1, end=10, value=1, step=.1, title="Amplitude")

callback = CustomJS(args=dict(source=source_all,
                              amp = amp_slider),
                    code="""
    const data = source.data;
    const A = amp.value;
    const x = data['x'];
    const y_original = data['y_left_original'];
    const y = data['y_left'];
    
    var mean = 0.0;
    for (var i = 0; i < x.length; i++) {
        mean += y_original[i];
    }
    mean = mean/x.length;
    for (var i = 0; i < x.length; i++) {
        y[i] = (y_original[i] - mean)*A + mean;
    }
            
    source.change.emit();
""")

amp_slider.js_on_change('value', callback)

# Select Tool!

In [18]:
if 0:
    select_tool = p.select(dict(type=BoxSelectTool))[0]

    source.callback = CustomJS(args=dict(p=p), code="""
            var inds = cb_obj.get('selected')['1d'].indices;
            var d1 = cb_obj.get('data');
            console.log(d1)
            var kernel = IPython.notebook.kernel;
            IPython.notebook.kernel.execute("inds = " + inds);
            """
    )
inds =""

In [19]:
p_grid_list = [[p_left, p_right],[p_range_tool,amp_slider]]
p = grid(p_grid_list)
show(p)

In [20]:
[source.data['x'][i] for i in inds]

[]

In [21]:
source_all.data['x']

DatetimeIndex(['2019-07-22 00:00:00', '2019-07-22 00:15:00',
               '2019-07-22 00:30:00', '2019-07-22 00:45:00',
               '2019-07-22 01:00:00', '2019-07-22 01:15:00',
               '2019-07-22 01:30:00', '2019-07-22 01:45:00',
               '2019-07-22 02:00:00', '2019-07-22 02:15:00',
               ...
               '2019-07-25 21:30:00', '2019-07-25 21:45:00',
               '2019-07-25 22:00:00', '2019-07-25 22:15:00',
               '2019-07-25 22:30:00', '2019-07-25 22:45:00',
               '2019-07-25 23:00:00', '2019-07-25 23:15:00',
               '2019-07-25 23:30:00', '2019-07-25 23:45:00'],
              dtype='datetime64[ns]', name='Timestamp', length=384, freq=None)

## Correlation plot



In [22]:
method ="spearman"
correlations = currencies_df.pct_change().corr(method = method)
N = correlations.shape[0]
#correlations = currencies_df.corr()
correlations

,EURUSD,GBPUSD,USDCHF,USDJPY,USDCNH,USDRUB,AUDUSD,NZDUSD,USDCAD,USDSEK,...,USDNOK,USDDKK,USDTRY,USDZAR,USDCZK,USDHUF,USDPLN,USDRUR,XAUUSD,XAGUSD
EURUSD,1.000000,0.533109,-0.657036,-0.342473,-0.295099,-0.099587,0.460679,0.464223,-0.339806,-0.633102,...,-0.578124,-0.986239,-0.142160,-0.159073,-0.833387,-0.856532,-0.831032,-0.033183,0.293165,0.249408
GBPUSD,0.533109,1.000000,-0.408184,-0.362685,-0.278719,-0.076432,0.373689,0.405201,-0.279635,-0.426821,...,-0.382709,-0.531468,-0.137924,-0.232621,-0.546863,-0.465440,-0.486013,0.021371,0.376748,0.279763
USDCHF,-0.657036,-0.408184,1.000000,0.407959,0.173566,0.047400,-0.357247,-0.373706,0.206074,0.417196,...,0.391555,0.666005,0.104166,0.128162,0.598614,0.612207,0.575675,0.006005,-0.323059,-0.266234
USDJPY,-0.342473,-0.362685,0.407959,1.000000,0.233097,0.039517,-0.263264,-0.357304,0.197947,0.247253,...,0.231680,0.335064,0.127349,0.177563,0.360007,0.336123,0.341896,-0.033507,-0.424968,-0.261615
USDCNH,-0.295099,-0.278719,0.173566,0.233097,1.000000,0.112098,-0.338090,-0.246270,0.233584,0.287505,...,0.340857,0.292836,0.175975,0.258641,0.305356,0.301601,0.319522,0.036153,-0.282984,-0.246520
USDRUB,-0.099587,-0.076432,0.047400,0.039517,0.112098,1.000000,-0.072398,-0.050275,0.141845,0.235052,...,0.268831,0.110247,0.080555,0.127309,0.114817,0.091586,0.130696,0.640781,-0.060784,-0.073850
AUDUSD,0.460679,0.373689,-0.357247,-0.263264,-0.338090,-0.072398,1.000000,0.688409,-0.434982,-0.451869,...,-0.476747,-0.465619,-0.204259,-0.228261,-0.456865,-0.468373,-0.467008,0.052566,0.283305,0.237023
NZDUSD,0.464223,0.405201,-0.373706,-0.357304,-0.246270,-0.050275,0.688409,1.000000,-0.384055,-0.429759,...,-0.432543,-0.453098,-0.128950,-0.245112,-0.459278,-0.443507,-0.462442,0.063768,0.320114,0.266629
USDCAD,-0.339806,-0.279635,0.206074,0.197947,0.233584,0.141845,-0.434982,-0.384055,1.000000,0.396649,...,0.470502,0.345208,0.157256,0.187751,0.359904,0.272498,0.339744,0.018992,-0.263082,-0.213907
USDSEK,-0.633102,-0.426821,0.417196,0.247253,0.287505,0.235052,-0.451869,-0.429759,0.396649,1.000000,...,0.676835,0.623489,0.074463,0.204472,0.619233,0.597427,0.590747,0.060140,-0.247268,-0.253230


In [23]:
correlations["EURUSD"]

EURUSD    1.000000
GBPUSD    0.533109
USDCHF   -0.657036
USDJPY   -0.342473
USDCNH   -0.295099
USDRUB   -0.099587
AUDUSD    0.460679
NZDUSD    0.464223
USDCAD   -0.339806
USDSEK   -0.633102
USDHKD    0.079321
USDSGD   -0.561337
USDMXN   -0.185462
USDNOK   -0.578124
USDDKK   -0.986239
USDTRY   -0.142160
USDZAR   -0.159073
USDCZK   -0.833387
USDHUF   -0.856532
USDPLN   -0.831032
USDRUR   -0.033183
XAUUSD    0.293165
XAGUSD    0.249408
Name: EURUSD, dtype: float64

In [24]:
correlations["EURUSD"]["USDTRY"]

-0.14216006642049914

In [25]:
# np.array(correlations).flatten()

array([ 1.        ,  0.53310864, -0.65703559, -0.34247328, -0.29509947,
       -0.09958666,  0.46067864,  0.46422322, -0.33980647, -0.63310183,
        0.07932101, -0.56133684, -0.18546169, -0.57812398, -0.98623875,
       -0.14216007, -0.15907329, -0.83338651, -0.85653162, -0.83103206,
       -0.03318334,  0.29316535,  0.24940807,  0.53310864,  1.        ,
       -0.40818356, -0.36268498, -0.27871865, -0.07643187,  0.37368893,
        0.40520089, -0.2796353 , -0.4268205 ,  0.14018729, -0.46414841,
       -0.21742289, -0.38270895, -0.53146832, -0.13792384, -0.23262103,
       -0.54686257, -0.46543972, -0.48601284,  0.02137103,  0.37674807,
        0.27976279, -0.65703559, -0.40818356,  1.        ,  0.40795941,
        0.17356559,  0.04740003, -0.35724705, -0.37370602,  0.20607437,
        0.41719612, -0.05513986,  0.46730657,  0.12589524,  0.39155508,
        0.66600505,  0.1041661 ,  0.12816188,  0.59861396,  0.61220742,
        0.57567535,  0.00600483, -0.32305858, -0.26623445, -0.34

In [39]:
import scipy.cluster.hierarchy as spc

corr = correlations.values
pdist = spc.distance.pdist(corr)
linkage = spc.linkage(pdist, method='complete')
group = spc.fcluster(linkage, 0.4 * pdist.max(), 'distance')
group

array([1, 1, 2, 3, 3, 4, 1, 1, 3, 2, 4, 2, 3, 2, 2, 3, 3, 2, 2, 2, 4, 1,
       1], dtype=int32)

In [40]:
group_indices_sort = group.argsort()
group_indices_sort

array([ 0, 21,  7,  6, 22,  1, 13, 19, 18, 17, 14, 11,  9,  2, 12,  4, 15,
       16,  8,  3, 20, 10,  5])

In [41]:
currencies = correlations.index.to_list()
sorted_currencies = np.array(currencies)[group_indices_sort]
sorted_currencies

array(['EURUSD', 'XAUUSD', 'NZDUSD', 'AUDUSD', 'XAGUSD', 'GBPUSD',
       'USDNOK', 'USDPLN', 'USDHUF', 'USDCZK', 'USDDKK', 'USDSGD',
       'USDSEK', 'USDCHF', 'USDMXN', 'USDCNH', 'USDTRY', 'USDZAR',
       'USDCAD', 'USDJPY', 'USDRUR', 'USDHKD', 'USDRUB'], dtype='<U6')

In [42]:
colormap = ["#444444", "#a6cee3", "#1f78b4", "#b2df8a", "#33a02c", "#fb9a99",
            "#e31a1c", "#fdbf6f", "#ff7f00", "#cab2d6", "#6a3d9a"]

xname = []
yname = []
color = []
alpha = []
groups_pair = []
correlation_array = []

for x_i in group_indices_sort:
    for y_i in group_indices_sort:
        x_currency = currencies[x_i]
        y_currency = currencies[y_i]
        xname.append(x_currency)
        yname.append(y_currency)

        alpha.append(min(np.abs(correlations[y_currency][x_currency]), 0.9) + 0.1)
        
        # Color the different clusters
        if group[x_i] == group[y_i]:
            color.append(colormap[group[x_i]%len(colormap)])
           # color.append('black')
        else:
            color.append('lightgray')
        
        groups_pair.append(str((group[x_i], group[y_i])))
        correlation_array.append(correlations[y_currency][x_currency])
data=dict(
    xname=xname,
    yname=yname,
    colors=color,
    alphas=alpha,
    groups_pair = groups_pair,
    correlation= correlation_array,
)

p = figure(title="Currency corrleations",
           x_axis_location="above", tools="hover,save",
           x_range=sorted_currencies , y_range= list(reversed(sorted_currencies)),
           tooltips = [('currencies', '@yname, @xname'), ('correlation', '@correlation'), ("groups","@groups_pair")])

p.plot_width = 600
p.plot_height = 600
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "10pt"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = np.pi/3

p.rect('xname', 'yname', 0.9, 0.9, source=data,
       color='colors', alpha='alphas', line_color=None,
       hover_line_color='black', hover_color='colors')

show(p) # show the plotn